# Vitessce Widget Tutorial

# Visualization of SpatialData Object

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from urllib.request import urlretrieve
import dask

dask.config.set({'dataframe.query-planning-warning': False})

from spatialdata import read_zarr
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    CoordinationLevel as CL,
    AbstractWrapper,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)
import zipfile


In [ ]:
zip_filepath = Path("data/visium.spatialdata.zarr.zip")
spatialdata_filepath = zip_filepath.with_suffix('')
if not zip_filepath.exists():
    spatialdata_filepath.parent.mkdir(exist_ok=True)
    urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_associated_xenium_io.zip', zip_filepath)
if not spatialdata_filepath.exists():
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(spatialdata_filepath.parent)
        (spatialdata_filepath.parent / "data.zarr").rename(spatialdata_filepath)

## 2. Load the data

In [ ]:
spatialdata = read_zarr(spatialdata_filepath)

In [ ]:
spatialdata

## 3. Create the Vitessce widget configuration

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.16",
    name='Visium SpatialData Demo (visium_associated_xenium_io)',
    description='From https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html'
)
# Add data to the configuration:
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/CytAssist_FFPE_Human_Breast_Cancer_full_image",
    table_path="table/table",
    obs_feature_matrix_path="table/table/X",
    obs_spots_path="shapes/CytAssist_FFPE_Human_Breast_Cancer",
    region="CytAssist_FFPE_Human_Breast_Cancer",
    coordinate_system="global",
    coordination_values={
        # The following tells Vitessce to consider each observation as a "spot"
        "obsType": "spot",
    }
)
dataset = vc.add_dataset(name='Breast Cancer Visium').add_object(wrapper)

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        'photometricInterpretation': 'RGB',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))
obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], [wrapper.obs_type_label])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller / obs_sets));

### 4. Render the widget

In [ ]:
vw = vc.widget()
vw